<a href="https://colab.research.google.com/github/mertege/FMCW-Data-Classification-/blob/main/Range_Doppler_CI4R_Datasets_Multi_Class_CNN_10Ghz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Image-based architecure is running with only 10GHz in Gürbüz dataset.
# 5-blocks are used. Reslotion 128x128
# Only Spectrogram plots are used.
# It was run 3-times. Results are shared on below:
# Min test accuracy is 0.887, Max test accuracy is 0.902, Mean test accuracy is 0.897, std of test accuracy is 0.007.
# Time elapsed during training time: 40.898 sec
# Time elapsed during test time: 0.179 sec

In [1]:
from keras.models import Sequential
from tensorflow.keras import layers
from keras.models import Model
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization, Normalization, Input, Conv2D, MaxPooling2D, Concatenate
from tensorflow.keras import regularizers
import tensorflow as tf
import scipy.io
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random
from numpy.random import seed
from sklearn.model_selection import KFold, StratifiedKFold
import time
from sklearn.metrics import precision_recall_fscore_support
from keras.callbacks import EarlyStopping
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras import backend as K 
import gc

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# Range-Doppler plots
bending_Xethru = scipy.io.loadmat('/content/drive/MyDrive/rangedopplers10ghz_bending.mat')
bending_Xethru = bending_Xethru['rds']
bending_Xethru = np.transpose(bending_Xethru, (2, 0, 1))
label_bending_Xethru           = np.zeros((bending_Xethru.shape[0],1))
label_bending_Xethru[:]        = 1

crawling_Xethru = scipy.io.loadmat('/content/drive/MyDrive/rangedopplers10ghz_bending.mat')
crawling_Xethru = crawling_Xethru['rds']
crawling_Xethru = np.transpose(crawling_Xethru, (2, 0, 1))
label_crawling_Xethru           = np.zeros((crawling_Xethru.shape[0],1))
label_crawling_Xethru[:]        = 2

kneeling_Xethru = scipy.io.loadmat('/content/drive/MyDrive/rangedopplers10ghz_kneeling.mat')
kneeling_Xethru = kneeling_Xethru['rds']
kneeling_Xethru = np.transpose(kneeling_Xethru, (2, 0, 1))
label_kneeling_Xethru           = np.zeros((kneeling_Xethru.shape[0],1))
label_kneeling_Xethru[:]        = 3

limping_Xethru = scipy.io.loadmat('/content/drive/MyDrive/rangedopplers10ghz_limping.mat')
limping_Xethru = limping_Xethru['rds']
limping_Xethru = np.transpose(limping_Xethru, (2, 0, 1))
label_limping_Xethru           = np.zeros((limping_Xethru.shape[0],1))
label_limping_Xethru[:]        = 4

pickingobject_Xethru = scipy.io.loadmat('/content/drive/MyDrive/rangedopplers10ghz_pickingobject.mat')
pickingobject_Xethru = pickingobject_Xethru['rds']
pickingobject_Xethru = np.transpose(pickingobject_Xethru, (2, 0, 1))
label_pickingobject_Xethru           = np.zeros((pickingobject_Xethru.shape[0],1))
label_pickingobject_Xethru[:]        = 5

scissorsgait_Xethru = scipy.io.loadmat('/content/drive/MyDrive/rangedopplers10ghz_scissorsgait.mat')
scissorsgait_Xethru = scissorsgait_Xethru['rds']
scissorsgait_Xethru = np.transpose(scissorsgait_Xethru, (2, 0, 1))
label_scissorsgait_Xethru           = np.zeros((scissorsgait_Xethru.shape[0],1))
label_scissorsgait_Xethru[:]        = 6

shortstep_Xethru = scipy.io.loadmat('/content/drive/MyDrive/rangedopplers10ghz_shortstep.mat')
shortstep_Xethru = shortstep_Xethru['rds']
shortstep_Xethru = np.transpose(shortstep_Xethru, (2, 0, 1))
label_shortstep_Xethru           = np.zeros((shortstep_Xethru.shape[0],1))
label_shortstep_Xethru[:]        = 7

sitting_Xethru = scipy.io.loadmat('/content/drive/MyDrive/rangedopplers10ghz_sitting.mat')
sitting_Xethru = sitting_Xethru['rds']
sitting_Xethru = np.transpose(sitting_Xethru, (2, 0, 1))
label_sitting_Xethru           = np.zeros((sitting_Xethru.shape[0],1))
label_sitting_Xethru[:]        = 8

walkingaway_Xethru = scipy.io.loadmat('/content/drive/MyDrive/rangedopplers10ghz_walkingaway.mat')
walkingaway_Xethru = walkingaway_Xethru['rds']
walkingaway_Xethru = np.transpose(walkingaway_Xethru, (2, 0, 1))
label_walkingaway_Xethru           = np.zeros((walkingaway_Xethru.shape[0],1))
label_walkingaway_Xethru[:]        = 9

walkingontoes_Xethru = scipy.io.loadmat('/content/drive/MyDrive/rangedopplers10ghz_walkingontoes.mat')
walkingontoes_Xethru = walkingontoes_Xethru['rds']
walkingontoes_Xethru = np.transpose(walkingontoes_Xethru, (2, 0, 1))
label_walkingontoes_Xethru           = np.zeros((walkingontoes_Xethru.shape[0],1))
label_walkingontoes_Xethru[:]        = 10

walkingtowards_Xethru = scipy.io.loadmat('/content/drive/MyDrive/rangedopplers10ghz_walkingtowards.mat')
walkingtowards_Xethru = walkingtowards_Xethru['rds']
walkingtowards_Xethru = np.transpose(walkingtowards_Xethru, (2, 0, 1))
label_walkingtowards_Xethru           = np.zeros((walkingtowards_Xethru.shape[0],1))
label_walkingtowards_Xethru[:]        = 11



In [ ]:
# Spectrogram plots
bending_Xethru_spectrogam = scipy.io.loadmat('/content/drive/MyDrive/spectrograms10ghz_bending.mat')
bending_Xethru_spectrogam = bending_Xethru_spectrogam['rds']
bending_Xethru_spectrogam = np.transpose(bending_Xethru_spectrogam, (2, 0, 1))
label_bending_Xethru_spectrogam           = np.zeros((bending_Xethru_spectrogam.shape[0],1))
label_bending_Xethru_spectrogam[:]        = 1

crawling_Xethru_spectrogam = scipy.io.loadmat('/content/drive/MyDrive/spectrograms10ghz_bending.mat')
crawling_Xethru_spectrogam = crawling_Xethru['rds']
crawling_Xethru_spectrogam = np.transpose(crawling_Xethru_spectrogam, (2, 0, 1))
label_crawling_Xethru_spectrogam           = np.zeros((crawling_Xethru_spectrogam.shape[0],1))
label_crawling_Xethru_spectrogam[:]        = 2

kneeling_Xethru_spectrogam = scipy.io.loadmat('/content/drive/MyDrive/spectrograms10ghz_kneeling.mat')
kneeling_Xethru_spectrogam = kneeling_Xethru_spectrogam['rds']
kneeling_Xethru_spectrogam = np.transpose(kneeling_Xethru_spectrogam, (2, 0, 1))
label_kneeling_Xethru_spectrogam           = np.zeros((kneeling_Xethru_spectrogam.shape[0],1))
label_kneeling_Xethru_spectrogam[:]        = 3

limping_Xethru_spectrogam = scipy.io.loadmat('/content/drive/MyDrive/spectrograms10ghz_limping.mat')
limping_Xethru_spectrogam = limping_Xethru_spectrogam['rds']
limping_Xethru_spectrogam = np.transpose(limping_Xethru_spectrogam, (2, 0, 1))
label_limping_Xethru_spectrogam           = np.zeros((limping_Xethru_spectrogam.shape[0],1))
label_limping_Xethru_spectrogam[:]        = 4

pickingobject_Xethru_spectrogam = scipy.io.loadmat('/content/drive/MyDrive/spectrograms10ghz_pickingobject.mat')
pickingobject_Xethru_spectrogam = pickingobject_Xethru_spectrogam['rds']
pickingobject_Xethru_spectrogam = np.transpose(pickingobject_Xethru_spectrogam, (2, 0, 1))
label_pickingobject_Xethru_spectrogam           = np.zeros((pickingobject_Xethru_spectrogam.shape[0],1))
label_pickingobject_Xethru_spectrogam[:]        = 5

scissorsgait_Xethru_spectrogam = scipy.io.loadmat('/content/drive/MyDrive/spectrograms10ghz_scissorsgait.mat')
scissorsgait_Xethru_spectrogam = scissorsgait_Xethru_spectrogam['rds']
scissorsgait_Xethru_spectrogam = np.transpose(scissorsgait_Xethru_spectrogam, (2, 0, 1))
label_scissorsgait_Xethru_spectrogam           = np.zeros((scissorsgait_Xethru_spectrogam.shape[0],1))
label_scissorsgait_Xethru_spectrogam[:]        = 6

shortstep_Xethru_spectrogam = scipy.io.loadmat('/content/drive/MyDrive/spectrograms10ghz_shortstep.mat')
shortstep_Xethru_spectrogam = shortstep_Xethru_spectrogam['rds']
shortstep_Xethru_spectrogam = np.transpose(shortstep_Xethru_spectrogam, (2, 0, 1))
label_shortstep_Xethru_spectrogam           = np.zeros((shortstep_Xethru_spectrogam.shape[0],1))
label_shortstep_Xethru_spectrogam[:]        = 7

sitting_Xethru_spectrogam = scipy.io.loadmat('/content/drive/MyDrive/spectrograms10ghz_sitting.mat')
sitting_Xethru_spectrogam = sitting_Xethru_spectrogam['rds']
sitting_Xethru_spectrogam = np.transpose(sitting_Xethru_spectrogam, (2, 0, 1))
label_sitting_Xethru_spectrogam           = np.zeros((sitting_Xethru_spectrogam.shape[0],1))
label_sitting_Xethru_spectrogam[:]        = 8

walkingaway_Xethru_spectrogam = scipy.io.loadmat('/content/drive/MyDrive/spectrograms10ghz_walkingaway.mat')
walkingaway_Xethru_spectrogam = walkingaway_Xethru_spectrogam['rds']
walkingaway_Xethru_spectrogam = np.transpose(walkingaway_Xethru_spectrogam, (2, 0, 1))
label_walkingaway_Xethru_spectrogam           = np.zeros((walkingaway_Xethru_spectrogam.shape[0],1))
label_walkingaway_Xethru_spectrogam[:]        = 9

walkingontoes_Xethru_spectrogam = scipy.io.loadmat('/content/drive/MyDrive/spectrograms10ghz_walkingontoes.mat')
walkingontoes_Xethru_spectrogam = walkingontoes_Xethru_spectrogam['rds']
walkingontoes_Xethru_spectrogam = np.transpose(walkingontoes_Xethru_spectrogam, (2, 0, 1))
label_walkingontoes_Xethru_spectrogam           = np.zeros((walkingontoes_Xethru_spectrogam.shape[0],1))
label_walkingontoes_Xethru_spectrogam[:]        = 10

walkingtowards_Xethru_spectrogam = scipy.io.loadmat('/content/drive/MyDrive/spectrograms10ghz_walkingtowards.mat')
walkingtowards_Xethru_spectrogam = walkingtowards_Xethru_spectrogam['rds']
walkingtowards_Xethru_spectrogam = np.transpose(walkingtowards_Xethru_spectrogam, (2, 0, 1))
label_walkingtowards_Xethru_spectrogam           = np.zeros((walkingtowards_Xethru_spectrogam.shape[0],1))
label_walkingtowards_Xethru_spectrogam[:]        = 11



In [18]:
spectrogram_concat = np.concatenate((bending_Xethru,crawling_Xethru),axis=0)
spectrogram_concat = np.concatenate((spectrogram_concat,kneeling_Xethru               ),axis=0)
spectrogram_concat = np.concatenate((spectrogram_concat,limping_Xethru  ),axis=0)
spectrogram_concat = np.concatenate((spectrogram_concat,pickingobject_Xethru   ),axis=0)
spectrogram_concat = np.concatenate((spectrogram_concat,scissorsgait_Xethru          ),axis=0)
spectrogram_concat = np.concatenate((spectrogram_concat,shortstep_Xethru            ),axis=0)
spectrogram_concat = np.concatenate((spectrogram_concat,sitting_Xethru                ),axis=0)
spectrogram_concat = np.concatenate((spectrogram_concat,walkingaway_Xethru),axis=0)
spectrogram_concat = np.concatenate((spectrogram_concat,walkingontoes_Xethru   ),axis=0)
spectrogram_concat = np.concatenate((spectrogram_concat,walkingtowards_Xethru  ),axis=0)

label_concat = np.concatenate((label_bending_Xethru,label_crawling_Xethru),axis=0)
label_concat = np.concatenate((label_concat,label_kneeling_Xethru               ),axis=0)
label_concat = np.concatenate((label_concat,label_limping_Xethru  ),axis=0)
label_concat = np.concatenate((label_concat,label_pickingobject_Xethru   ),axis=0)
label_concat = np.concatenate((label_concat,label_scissorsgait_Xethru          ),axis=0)
label_concat = np.concatenate((label_concat,label_shortstep_Xethru           ),axis=0)
label_concat = np.concatenate((label_concat,label_sitting_Xethru                ),axis=0)
label_concat = np.concatenate((label_concat,label_walkingaway_Xethru),axis=0)
label_concat = np.concatenate((label_concat,label_walkingontoes_Xethru  ),axis=0)
label_concat = np.concatenate((label_concat,label_walkingtowards_Xethru ),axis=0)

In [ ]:
spectrogram_concat_spectrogam = np.concatenate((bending_Xethru_spectrogam,crawling_Xethru_spectrogam),axis=0)
spectrogram_concat_spectrogam = np.concatenate((spectrogram_concat_spectrogam,kneeling_Xethru_spectrogam               ),axis=0)
spectrogram_concat_spectrogam = np.concatenate((spectrogram_concat_spectrogam,limping_Xethru_spectrogam  ),axis=0)
spectrogram_concat_spectrogam = np.concatenate((spectrogram_concat_spectrogam,pickingobject_Xethru_spectrogam   ),axis=0)
spectrogram_concat_spectrogam = np.concatenate((spectrogram_concat_spectrogam,scissorsgait_Xethru_spectrogam          ),axis=0)
spectrogram_concat_spectrogam = np.concatenate((spectrogram_concat_spectrogam,shortstep_Xethru_spectrogam            ),axis=0)
spectrogram_concat_spectrogam = np.concatenate((spectrogram_concat_spectrogam,sitting_Xethru_spectrogam                ),axis=0)
spectrogram_concat_spectrogam = np.concatenate((spectrogram_concat_spectrogam,walkingaway_Xethru_spectrogam),axis=0)
spectrogram_concat_spectrogam = np.concatenate((spectrogram_concat_spectrogam,walkingontoes_Xethru_spectrogam   ),axis=0)
spectrogram_concat_spectrogam = np.concatenate((spectrogram_concat_spectrogam,walkingtowards_Xethru_spectrogam  ),axis=0)

label_concat_spectrogam = np.concatenate((label_bending_Xethru_spectrogam,label_crawling_Xethru_spectrogam),axis=0)
label_concat_spectrogam = np.concatenate((label_concat_spectrogam,label_kneeling_Xethru_spectrogam               ),axis=0)
label_concat_spectrogam = np.concatenate((label_concat_spectrogam,label_limping_Xethru_spectrogam  ),axis=0)
label_concat_spectrogam = np.concatenate((label_concat_spectrogam,label_pickingobject_Xethru_spectrogam   ),axis=0)
label_concat_spectrogam = np.concatenate((label_concat_spectrogam,label_scissorsgait_Xethru_spectrogam          ),axis=0)
label_concat_spectrogam = np.concatenate((label_concat_spectrogam,label_shortstep_Xethru_spectrogam           ),axis=0)
label_concat_spectrogam = np.concatenate((label_concat_spectrogam,label_sitting_Xethru_spectrogam                ),axis=0)
label_concat_spectrogam = np.concatenate((label_concat_spectrogam,label_walkingaway_Xethru_spectrogam),axis=0)
label_concat_spectrogam = np.concatenate((label_concat_spectrogam,label_walkingontoes_Xethru_spectrogam  ),axis=0)
label_concat_spectrogam = np.concatenate((label_concat_spectrogam,label_walkingtowards_Xethru_spectrogam ),axis=0)

In [ ]:
del bending_Xethru
del crawling_Xethru
del kneeling_Xethru
del limping_with_RL_Stiff_Xethru
del picking_up_an_object_Xethru
del scissors_gait_Xethru
del short_steps_Xethru
del sitting_Xethru
del walking_away_from_Radar_Xethru
del Walking_on_both_toes_Xethru
del Walking_towards_radar_Xethru


In [19]:
# One-Hot Encode Label
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(label_concat)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded_labels = onehot_encoder.fit_transform(integer_encoded)
spectrogram_concat_label = onehot_encoded_labels

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [20]:
# Shuffle concat range doppler
shuffle_indx = random.sample(range(0, spectrogram_concat.shape[0]), spectrogram_concat.shape[0]) # split validation data
spectrogram_concat = spectrogram_concat[:,:,:, np.newaxis]
spectrogram_concat_shuffle = spectrogram_concat[shuffle_indx,:,:,:]
spectrogram_concat_label_shuffle = spectrogram_concat_label[shuffle_indx,:]
del spectrogram_concat
del spectrogram_concat_label

In [21]:
range_doppler_concat_shuffle = spectrogram_concat_shuffle
del spectrogram_concat_shuffle
range_doppler_concat_label = label_concat
del label_concat
range_doppler_concat_label_shuffle = range_doppler_concat_label[shuffle_indx,:]

In [22]:
# ---------------- Augmente and shuffle (train and test) data data ----------------
def mixup_augmentation(range_doppler_training_data, labels, repeat_of_mixup, alpha=0.2):
    batch_size = range_doppler_training_data.shape[0]
    concat_images = np.zeros((batch_size*(repeat_of_mixup+1),range_doppler_training_data.shape[1],range_doppler_training_data.shape[2],range_doppler_training_data.shape[3]))
    concat_images_range_doppler = np.zeros((batch_size*(repeat_of_mixup+1),range_doppler_training_data.shape[1],range_doppler_training_data\
                                            .shape[2],range_doppler_training_data.shape[3]))
    concat_label = np.zeros((batch_size*(repeat_of_mixup+1),labels.shape[1]))
    
    for ii in range(repeat_of_mixup):
      # shuffle train dataset
      shuffle_indx_1 = random.sample(range(0, range_doppler_training_data.shape[0]), range_doppler_training_data.shape[0]) # split validation data
      images_shuffled_1 = range_doppler_training_data[shuffle_indx_1,:,:,:]
      range_doppler_training_data_shuffled_1 = range_doppler_training_data[shuffle_indx_1,:,:,:]
      labels_shuffled_1 = labels[shuffle_indx_1,:]

      shuffle_indx_2 = random.sample(range(0, range_doppler_training_data.shape[0]), range_doppler_training_data.shape[0]) # split validation data
      images_shuffled_2 = range_doppler_training_data[shuffle_indx_2,:,:,:]
      range_doppler_training_data_shuffled_2 = range_doppler_training_data[shuffle_indx_2,:,:,:]
      labels_shuffled_2 = labels[shuffle_indx_2,:]

      # Sample lambda and reshape it to do the mixup
      ll = np.random.beta(alpha, alpha, (batch_size,1,1,1))
      x_l = np.reshape(ll, (batch_size,1,1,1))
      y_l = np.reshape(ll, (batch_size,1))
      
      # Perform mixup on both images and labels by combining a pair of images/labels
      images_mixup_range_doppler = range_doppler_training_data_shuffled_1 * x_l + range_doppler_training_data_shuffled_2 * (1 - x_l)
      labels_mixup = labels_shuffled_1 * y_l + labels_shuffled_2 * (1 - y_l)
      concat_images_range_doppler[ii*batch_size:(ii+1)*batch_size,:,:,:] = images_mixup_range_doppler
      concat_label[ii*batch_size:(ii+1)*batch_size,:] = labels_mixup

    concat_images_range_doppler[repeat_of_mixup*batch_size:,:,:,:] = range_doppler_training_data
    concat_label[repeat_of_mixup*batch_size:,:] = labels
    return (concat_images_range_doppler, concat_label)

def split_and_augmentation_of_training(range_doppler_concat_shuffle_train,range_doppler_concat_label_shuffle_train,\
                                       repeat_of_mixup, augmentation_enable):
  # ---------------- Parameters ----------------
  size_of_validation = 50
  # Split validation
  randomlist_for_validation_indx = random.sample(range(0, range_doppler_concat_shuffle_train.shape[0]), size_of_validation) # split validation data
  randomlist_for_train_indx = np.delete(range(0, range_doppler_concat_shuffle_train.shape[0]), randomlist_for_validation_indx) # split training data
  # get validation data
  spectrogram_validation_labels = range_doppler_concat_label_shuffle_train[randomlist_for_validation_indx,:]
  range_doppler_validation_data = range_doppler_concat_shuffle_train[randomlist_for_validation_indx,:,:,:]
  # get training data
  spectrogram_training_labels = spectrogram_concat_label_shuffle_train[randomlist_for_train_indx,:]
  range_doppler_training_data = range_doppler_concat_shuffle_train[randomlist_for_train_indx,:,:,:]

  (range_doppler_augmented_image,spectrograms_label)=\
   mixup_augmentation(range_doppler_training_data, spectrogram_training_labels, repeat_of_mixup, alpha=0.2)

  return (range_doppler_augmented_image,spectrograms_label,\
     range_doppler_validation_data, spectrogram_validation_labels)



def normalize_inputs(range_doppler_concat_shuffle, normalize_inputs_enable):
  # ---------------- Normalize Inputs ----------------
  if normalize_inputs_enable == True:
    layer = Normalization(axis=None)
    layer.adapt(range_doppler_concat_shuffle)
    range_doppler_concat_shuffle = layer(range_doppler_concat_shuffle)
  else:
    range_doppler_concat_shuffle = range_doppler_concat_shuffle
  return(range_doppler_concat_shuffle)


In [23]:
range_doppler_concat_shuffle_new = np.zeros((range_doppler_concat_shuffle.shape[0],128,128,1))
for ii in range(range_doppler_concat_shuffle.shape[0]):
  aa = range_doppler_concat_shuffle[ii,:,:,0]
  range_doppler_concat_shuffle_new[ii,:,:,0] = cv2.resize(aa, (range_doppler_concat_shuffle_new.shape[2], range_doppler_concat_shuffle_new.shape[1]),interpolation = cv2.INTER_CUBIC)
range_doppler_concat_shuffle = range_doppler_concat_shuffle_new
del aa
del range_doppler_concat_shuffle_new

In [24]:
normalize_inputs_enable = 1
range_doppler_concat_shuffle = normalize_inputs(range_doppler_concat_shuffle, normalize_inputs_enable)
range_doppler_concat_shuffle = np.float32(range_doppler_concat_shuffle)


In [25]:

# ---------- Parameters ----------------
augmentation_enable = True
normalize_inputs_enable = True
num_folds = 5
kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state = None) # random_state = 1 ile split run'dan run'a sabit.
test_accuracy_per_run = []
f1_score_per_run = []
epoch_number = 100
batch_size = 32
dense_size = 128
dropout_prob_cnn = 0.1
dropout_prob_dense = 0.3
repeat_of_mixup = 2
number_of_repeat = 3
for repeat_run_number in range(number_of_repeat):
  test_accuracy_per_fold = []
  f1_score_per_fold = []
  if repeat_run_number > 0:
    del range_doppler_concat_shuffle_test
    del range_doppler_augmented_image
    del range_doppler_concat_shuffle_train
   
  for randomlist_for_train_indx, randomlist_for_test_indx in kfold.split(range_doppler_concat_shuffle,range_doppler_concat_label_shuffle):   
    gc.collect()
    K.clear_session()
    
    # test data
    range_doppler_concat_shuffle_test = range_doppler_concat_shuffle[randomlist_for_test_indx,:,:,:]
    range_doppler_concat_label_shuffle_test = spectrogram_concat_label_shuffle[randomlist_for_test_indx,:]
    #train data
    range_doppler_concat_shuffle_train = range_doppler_concat_shuffle[randomlist_for_train_indx,:,:,:]
    spectrogram_concat_label_shuffle_train = spectrogram_concat_label_shuffle[randomlist_for_train_indx,:]
      # ---------------- MixUp Augmentation ----------------
    (range_doppler_augmented_image,spectrogram_concat_label_shuffle_concat,\
     validation_range_doppler, spectrogram_validation_labels)  =\
      split_and_augmentation_of_training(range_doppler_concat_shuffle_train,\
                                         spectrogram_concat_label_shuffle_train,\
                                         repeat_of_mixup, augmentation_enable)
    
    # ---------------- Neural Network Architecture ----------------

    def encoder_for_range_doppler(input_shape):
      input = Input(shape=input_shape)
      x = Conv2D(4, (3, 3),padding='same',kernel_initializer="glorot_normal")(input)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
      x = Dropout(dropout_prob_cnn)(x)
      x = MaxPooling2D(pool_size=(2, 2))(x)
      x = Conv2D(8, (3, 3),padding='same',kernel_initializer="glorot_normal")(x)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
      x = Dropout(dropout_prob_cnn)(x)
      x = MaxPooling2D(pool_size=(2, 2))(x)
      x = Conv2D(16, (3, 3),padding='same',kernel_initializer="glorot_normal")(x)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
      x = Dropout(0.2)(x)
      x = MaxPooling2D(pool_size=(2, 2))(x)   
      x = Conv2D(16, (3, 3),padding='same',kernel_initializer="glorot_normal")(x)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
      x = Dropout(0.2)(x)
      x = MaxPooling2D(pool_size=(2, 2))(x)     
      x = Conv2D(16, (3, 3),padding='same',kernel_initializer="glorot_normal")(x)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
      x = Dropout(0.2)(x)
      x = MaxPooling2D(pool_size=(2, 2))(x)           
      x = Flatten()(x)
      return Model(input, x)


    def decoder_for_concat(input_shape):
      input = Input(shape=input_shape)
      x = Dense(128,kernel_regularizer='l2')(input)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
      x = Dropout(0.5)(x)
      x = Dense(dense_size,kernel_regularizer='l2')(x)
      # x = BatchNormalization()(x)
      x = Activation('relu')(x)
      x = Dropout(dropout_prob_dense)(x)
      x = Dense(11, activation="softmax")(x)
      return Model(input, x)
    input_shape = range_doppler_concat_shuffle.shape[1:]
    base_network_range_doppler = encoder_for_range_doppler(input_shape)
    range_doppler_input  = Input(shape=input_shape)
    processed_range_doppler  = base_network_range_doppler(range_doppler_input)


    base_decoder_network = decoder_for_concat((processed_range_doppler.shape[1]))
    out = base_decoder_network(processed_range_doppler)

    model = Model(inputs=[range_doppler_input], outputs=[out]) 
    print(model.summary())
    # ---------------- Compile and Fit ----------------
    model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    
    earlyStopping = EarlyStopping(monitor='val_loss', patience=35, verbose=0,restore_best_weights=True, mode='min')
    t = time.time()
    history = model.fit((range_doppler_augmented_image),(spectrogram_concat_label_shuffle_concat),
                    epochs=epoch_number,
                    batch_size=batch_size,
                    shuffle = True,
                    verbose=0,
                    callbacks=[earlyStopping],
                    validation_data = ((validation_range_doppler) , (spectrogram_validation_labels)))
    elapsed = time.time() - t
    test_loss, test_accuracy  = model.evaluate([range_doppler_concat_shuffle_test],\
                                               [range_doppler_concat_label_shuffle_test],
                  batch_size=batch_size)

    test_accuracy_per_fold.append(test_accuracy)
    # f1_score_per_fold.append(test_f1_score)

  test_accuracy_per_run.append(sum(test_accuracy_per_fold)/num_folds)
  # f1_score_per_run.append(sum(f1_score_per_fold)/num_folds)
  print(test_accuracy_per_run)
  # print(f1_score_per_run)
print(f'Mean test accuracy is {"{:.2f}".format(sum(test_accuracy_per_run)/number_of_repeat)},\
max test accuracy is {"{:.2f}".format(max(test_accuracy_per_run))},\
min test accuracy is {"{:.2f}".format(min(test_accuracy_per_run))}, \
std of test accuracy is {"{:.2f}".format(np.std(test_accuracy_per_run, axis=0))}.')
print(f'Time elapsed through all process: {"{:.2f}".format(elapsed)}, sec')

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 1)]     0         
                                                                 
 model (Functional)          (None, 256)               6384      
                                                                 
 model_1 (Functional)        (None, 11)                51339     
                                                                 
Total params: 57,723
Trainable params: 57,347
Non-trainable params: 376
_________________________________________________________________
None
5/5 [==============================] - 0s 48ms/step - loss: 0.4120 - accuracy: 0.8992
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 1)]     0         
           

In [26]:
t_test = time.time()
y_test_predicted = model.predict((range_doppler_concat_shuffle_test[0:1,:,:,:]))
test_elapsed = time.time() - t_test 
print(f'Time elapsed during test time: {"{:.2f}".format(test_elapsed)}, sec')

Time elapsed during test time: 0.18, sec


In [27]:
print(f'Min test accuracy is {"{:.3f}".format(min(test_accuracy_per_run))}, \
Max test accuracy is {"{:.3f}".format(max(test_accuracy_per_run))}, \
Mean test accuracy is {"{:.3f}".format(sum(test_accuracy_per_run)/number_of_repeat)}, \
std of test accuracy is {"{:.3f}".format(np.std(test_accuracy_per_run, axis=0))}.')
print(f'Time elapsed during training time: {"{:.3f}".format(elapsed)} sec')
print(f'Time elapsed during test time: {"{:.3f}".format(test_elapsed)} sec')

Min test accuracy is 0.887, Max test accuracy is 0.902, Mean test accuracy is 0.897, std of test accuracy is 0.007.
Time elapsed during training time: 40.898 sec
Time elapsed during test time: 0.179 sec
